In [1]:
from scipy import optimize, stats
import numpy as np
import timeit

## Examples

In [2]:
def mk_quad(epsilon, ndim=2):
  def f(x):
    x = np.asarray(x)
    y = x.copy()
    y *= np.power(epsilon, np.arange(ndim))
    return .33*np.sum(y**2)
  
  def gradient(x):
    x = np.asarray(x)
    y = x.copy()
    scaling = np.power(epsilon, np.arange(ndim))
    y *= scaling
    return .33*2*scaling*y
  
  def hessian(x):
    scaling = np.power(epsilon, np.arange(ndim))
    return .33*2*np.diag(scaling)
  
  return f, gradient, hessian

In [3]:
import cProfile

f, grad, hess = mk_quad(0.7)
def run():
  for i in range(100):
    optimize.minimize(fun = f, x0 = (1.6, 1.1), jac=grad, method="BFGS", tol=1e-11)

cProfile.run('run()', sort="tottime")

         128904 function calls in 0.192 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.032    0.000    0.187    0.002 _optimize.py:1318(_minimize_bfgs)
    15200    0.016    0.000    0.068    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     5800    0.016    0.000    0.016    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      900    0.009    0.000    0.092    0.000 _linesearch.py:91(scalar_search_wolfe1)
     2600    0.008    0.000    0.032    0.000 _optimize.py:235(vecnorm)
     1000    0.007    0.000    0.009    0.000 2689226355.py:8(gradient)
     1000    0.007    0.000    0.020    0.000 2689226355.py:2(f)
     3700    0.007    0.000    0.021    0.000 fromnumeric.py:69(_wrapreduction)
     2000    0.006    0.000    0.015    0.000 numeric.py:2407(array_equal)
      900    0.005    0.000    0.035    0.000 _linesearch.py:77(derphi)
     5100    0.004    0.000    0.014    

## Exercise 1

In [4]:
def f(x):
    return np.exp(x[0]-1) + np.exp(-x[1]+1)+(x[0]-x[1])**2

def grad(x):
    return [
      np.exp(x[0]-1) + 2*(x[0]-x[1]),
      -np.exp(-x[1]+1) - 2*(x[0]-x[1])
    ]

x0 = [0, 0]

In [5]:
optimize.minimize(fun=f, x0=x0, jac=grad, method="bfgs")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1.7973886823709806
        x: [ 7.961e-01  1.204e+00]
      nit: 7
      jac: [ 9.113e-06 -2.533e-06]
 hess_inv: [[ 6.736e-01  4.859e-01]
            [ 4.859e-01  7.045e-01]]
     nfev: 8
     njev: 8

In [6]:
optimize.minimize(fun=f, x0=x0, jac=grad, method="cg")

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 1.7973886823532366
       x: [ 7.961e-01  1.204e+00]
     nit: 9
     jac: [-3.168e-06  7.683e-07]
    nfev: 15
    njev: 15

In [7]:
optimize.minimize(fun=f, x0=x0, jac=grad, method="newton-cg")

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 1.7973886823506673
       x: [ 7.961e-01  1.204e+00]
     nit: 7
     jac: [-7.069e-09  1.138e-08]
    nfev: 8
    njev: 22
    nhev: 0

In [8]:
optimize.minimize(fun=f, x0=x0, method="nelder-mead")

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.797388682785636
             x: [ 7.961e-01  1.204e+00]
           nit: 67
          nfev: 128
 final_simplex: (array([[ 7.961e-01,  1.204e+00],
                       [ 7.961e-01,  1.204e+00],
                       [ 7.961e-01,  1.204e+00]]), array([ 1.797e+00,  1.797e+00,  1.797e+00]))

In [9]:
[
  timeit.Timer( lambda: optimize.minimize(fun=f, x0=x0, jac=grad, method="bfgs") ).repeat(1,100),
  timeit.Timer( lambda: optimize.minimize(fun=f, x0=x0, jac=grad, method="cg") ).repeat(1,100),
  timeit.Timer( lambda: optimize.minimize(fun=f, x0=x0, jac=grad, method="newton-cg") ).repeat(1,100),
  timeit.Timer( lambda: optimize.minimize(fun=f, x0=x0, method="nelder-mead") ).repeat(1,100)
]

[[0.09146437514573336],
 [0.10230429517105222],
 [0.1239176681265235],
 [0.3147982959635556]]

## Exercise 2

In [10]:
from scipy.stats import gamma

g = gamma(a=2.0, scale=2.0)
x = g.rvs(size=100, random_state=1234)
x.round(2)

array([ 4.7 ,  1.11,  1.8 ,  6.19,  3.37,  0.25,  6.45,  0.36,  4.49,
        4.14,  2.84,  1.91,  8.03,  2.26,  2.88,  6.88,  6.84,  6.83,
        6.1 ,  3.03,  3.67,  2.57,  3.53,  2.07,  4.01,  1.51,  5.69,
        3.92,  6.01,  0.82,  2.11,  2.97,  5.02,  9.13,  4.19,  2.82,
       11.81,  1.17,  1.69,  4.67,  1.47, 11.67,  5.25,  3.44,  8.04,
        3.74,  5.73,  6.58,  3.54,  2.4 ,  1.32,  2.04,  2.52,  4.89,
        4.14,  5.02,  4.75,  8.24,  7.6 ,  1.  ,  6.14,  0.58,  2.83,
        2.88,  5.42,  0.5 ,  3.46,  4.46,  1.86,  4.59,  2.24,  2.62,
        3.99,  3.74,  5.27,  1.42,  0.56,  7.54,  5.5 ,  1.58,  5.49,
        6.57,  4.79,  5.84,  8.21,  1.66,  1.53,  4.27,  2.57,  1.48,
        5.23,  3.84,  3.15,  2.1 ,  3.71,  2.79,  0.86,  8.52,  4.36,
        3.3 ])

In [11]:
def mle_gamma(θ): 
  if θ[0] <= 0 or θ[1] <= 0:
    return 1e16
  else:
    return -np.sum(gamma.logpdf(x, a=θ[0], scale=θ[1]))

In [12]:
mle_gamma([1,1])

400.7016808737107

In [13]:
optimize.minimize(
  mle_gamma, x0=[1,1], method="bfgs"
)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 221.38372813334652
        x: [ 2.411e+00  1.662e+00]
      nit: 11
      jac: [-9.537e-06 -5.722e-06]
 hess_inv: [[ 1.015e-01 -7.057e-02]
            [-7.057e-02  6.022e-02]]
     nfev: 57
     njev: 19

In [14]:
optimize.minimize(
  mle_gamma, x0=[1,1], method="l-bfgs-b",
  bounds=[(1e-8,1e8),(1e-8,1e8)]
)

  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 221.383728133346
        x: [ 2.411e+00  1.662e+00]
      nit: 12
      jac: [-2.842e-06  0.000e+00]
     nfev: 51
     njev: 17
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>

In [15]:
gamma.fit(x, floc=0)

(2.411169989586957, 0, 1.6618557903599012)